<a href="https://colab.research.google.com/github/daycardoso/PredictCost/blob/main/PredicCostRandomForestRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
# Carregar o datset unificado
df_unificado = pd.read_csv('/content/drive/MyDrive/Trabalho ML Mestrado 01-2025/df_unificado.csv')

# matriz contendo os atributos
X = df_unificado.iloc[:, :-1].values

# vetor contendo o custo, ou seja, a ultima coluna
y = df_unificado.iloc[:, -1].values

# nome de cada atributo
feature_names = df_unificado.columns[:-1]

# nome de cada classe
target_names = df_unificado.columns[-1]

print(f"Dimensões de X: {X.shape}\n")
print(f"Dimensões de y: {y.shape}\n")
print(f"Nomes dos atributos: {feature_names}\n")
print(f"Nomes das classes: {target_names}")

Dimensões de X: (6336328, 8)

Dimensões de y: (6336328,)

Nomes dos atributos: Index(['x0', 'y0', 'theta0', 'beta0', 'xf', 'yf', 'thetaf', 'betaf'], dtype='object')

Nomes das classes: cost


In [6]:
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_validate

# 1) Cria um hold-out antes de qualquer CV
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

pipeline = Pipeline([
    ('regressor', RandomForestRegressor(random_state=42,
                                        criterion='absolute_error',
                                        max_depth=20,
                                        n_jobs=-1))
    ])


In [ ]:
from sklearn.model_selection import RepeatedKFold

# 5×5 CV repetida: balanceia viés x variância na estimação
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)


In [ ]:
from sklearn.model_selection import cross_validate

scoring = {
    'R2': 'r2',
    'MSE': 'neg_mean_squared_error',
    'MAE': 'neg_mean_absolute_error',
    'MAPE': 'neg_mean_absolute_percentage_error',
    'MedAE': 'neg_median_absolute_error',
    'MaxE': 'max_error',
    'EVS': 'explained_variance',
}

cv_results = cross_validate(
    pipeline, X_train_full, y_train_full,
    cv=cv, scoring=scoring, return_train_score=True, n_jobs=-1
)

In [ ]:

# Treinar o modelo
# pipeline.fit(X_train_full, y_train_full)
modelo = pipeline.fit(X_train_full, y_train_full)